# Comprehensive D&D Model Training Notebook

This notebook provides an interactive interface for training custom D&D models.

## Features
- Automated Environment Setup
- Data Analysis and Visualization
- Interactive Training Configuration
- Model Training and Validation
- GGUF Export for CactusTTS
- Complete Pipeline Automation

In [ ]:
# Setup and imports
import sys
import os
import json
from pathlib import Path
from datetime import datetime

# Add parent directory for imports
sys.path.append('..')

print("🚀 Notebook environment initialized")
print(f"📁 Working directory: {os.getcwd()}")
print(f"🐍 Python version: {sys.version}")

In [ ]:
# Environment validation
def check_environment():
    """Check if environment is ready for training."""
    results = {'pytorch': False, 'transformers': False, 'gpu_available': False}
    
    try:
        import torch
        results['pytorch'] = True
        print(f"✅ PyTorch {torch.__version__} detected")
        
        if torch.cuda.is_available():
            results['gpu_available'] = True
            print(f"✅ CUDA GPU: {torch.cuda.get_device_name()}")
        elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
            results['gpu_available'] = True
            print("✅ Apple Silicon GPU available")
        else:
            print("⚠️  CPU training (slower)")
    except ImportError:
        print("❌ PyTorch not found")
    
    try:
        import transformers
        results['transformers'] = True
        print(f"✅ Transformers {transformers.__version__}")
    except ImportError:
        print("❌ Transformers not found")
    
    return results

env_status = check_environment()
print(f"\n📊 Environment Status: {sum(env_status.values())}/{len(env_status)} checks passed")

In [ ]:
# Training data analysis
def analyze_training_data():
    """Analyze training data structure."""
    data_dir = Path("../data/scenarios")
    
    if not data_dir.exists():
        print("❌ No training data directory found")
        return None
    
    md_files = list(data_dir.rglob("*.md"))
    print(f"📖 Found {len(md_files)} training files")
    
    analysis = {
        'total_files': len(md_files),
        'categories': {},
        'tool_calls': {},
        'conversations': 0
    }
    
    import re
    tool_pattern = r'\[([a-zA-Z_][a-zA-Z0-9_]*)\s*:\s*([^\]]+)\]'
    
    for md_file in md_files:
        category = md_file.parent.name
        analysis['categories'][category] = analysis['categories'].get(category, 0) + 1
        
        try:
            with open(md_file, 'r', encoding='utf-8') as f:
                content = f.read()
            
            sections = content.split('\n# ')
            analysis['conversations'] += max(0, len(sections) - 1)
            
            tool_matches = re.findall(tool_pattern, content)
            for tool_name, args in tool_matches:
                tool_name = tool_name.lower()
                analysis['tool_calls'][tool_name] = analysis['tool_calls'].get(tool_name, 0) + 1
                
        except Exception as e:
            print(f"⚠️  Error analyzing {md_file}: {e}")
    
    return analysis

data_analysis = analyze_training_data()

if data_analysis:
    print(f"\n📊 Training Data Analysis:")
    print(f"  📁 Total files: {data_analysis['total_files']}")
    print(f"  💬 Conversations: {data_analysis['conversations']}")
    print(f"  🔧 Unique tools: {len(data_analysis['tool_calls'])}")
    
    print(f"\n📂 Categories:")
    for category, count in data_analysis['categories'].items():
        print(f"  - {category}: {count} files")
    
    print(f"\n🔧 Top Tools:")
    top_tools = sorted(data_analysis['tool_calls'].items(), key=lambda x: x[1], reverse=True)[:5]
    for tool, count in top_tools:
        print(f"  - {tool}: {count} uses")

In [ ]:
# Model training function
def run_training(mode='full', learning_rate=2e-4, max_steps=60):
    """Execute model training."""
    from train_dnd_model_v2 import DNDModelTrainer
    
    trainer = DNDModelTrainer()
    trainer.incremental_mode = (mode == 'incremental')
    trainer.learning_rate = learning_rate
    trainer.max_steps = max_steps
    
    if trainer.incremental_mode:
        trainer.base_model_path = './trained_models/dnd_model'
        print(f"🔄 Incremental training mode")
    else:
        print(f"🆕 Full training mode")
    
    print(f"⚙️  Configuration:")
    print(f"  📊 Learning rate: {trainer.learning_rate}")
    print(f"  🔢 Max steps: {trainer.max_steps}")
    
    start_time = datetime.now()
    print(f"\n🚀 Starting training at {start_time.strftime('%H:%M:%S')}...")
    
    success = trainer.train()
    
    end_time = datetime.now()
    duration = end_time - start_time
    
    if success:
        print(f"\n✅ Training completed successfully in {duration}")
    else:
        print(f"\n❌ Training failed after {duration}")
    
    return success

print("🏋️‍♂️ Training function ready")
print("Use: run_training(mode='full', learning_rate=2e-4, max_steps=60)")

In [ ]:
# Tool extension
def extend_tools():
    """Extend tool vocabulary."""
    from tool_extension_framework import ToolExtensionFramework
    
    framework = ToolExtensionFramework("../data", "../trained_models")
    success = framework.run_tool_extension(auto_approve=True)
    
    if success:
        print("✅ Tool vocabulary extended!")
    else:
        print("❌ Tool extension failed")
    
    return success

print("🔧 Tool extension function ready")
print("Use: extend_tools()")

In [ ]:
# Complete pipeline
def run_complete_pipeline():
    """Run the complete training pipeline."""
    print("🚀 Starting Complete Training Pipeline")
    print("=" * 50)
    
    steps = [
        ("🔧 Tool Extension", extend_tools),
        ("🏋️‍♂️ Model Training", lambda: run_training()),
    ]
    
    results = {}
    
    for step_name, step_func in steps:
        print(f"\n{step_name}...")
        try:
            success = step_func()
            results[step_name] = success
            if success:
                print(f"✅ {step_name} completed")
            else:
                print(f"❌ {step_name} failed")
                break
        except Exception as e:
            print(f"❌ {step_name} error: {e}")
            results[step_name] = False
            break
    
    print("\n" + "=" * 60)
    print("🎉 PIPELINE COMPLETED!")
    print("=" * 60)
    
    for step_name, success in results.items():
        status = "✅" if success else "❌"
        print(f"{status} {step_name}")
    
    all_success = all(results.values())
    if all_success:
        print("\n🎊 All steps completed successfully!")
        print("Your D&D model is ready for CactusTTS integration!")
    else:
        print("\n⚠️  Some steps failed. Check the output above.")
    
    return all_success

print("🎯 Complete pipeline function ready")
print("Use: run_complete_pipeline()")

## Usage Examples

### Quick Start
```python
# Run complete pipeline
run_complete_pipeline()
```

### Individual Steps
```python
# Extend tools
extend_tools()

# Train model
run_training(mode='full', learning_rate=2e-4, max_steps=60)

# Incremental training
run_training(mode='incremental', learning_rate=1e-6, max_steps=20)
```

### Integration
After training, your model will be available at:
- **Model**: `./trained_models/dnd_model/`
- **Config**: `./trained_models/dnd_model/cactus_config.json`
- **Tools**: `./trained_models/tool_registry.json`